# aws data analytics toolkit

## get base path

In [1]:
import os
base = os.getcwd()
print(base)

base_split = base.split("/")
base_split

new_base_split = base_split[:5]
# new_base = os.path.join(new_base_split)
print(new_base_split)

new_base_join = [s for s in new_base_split]
new_base_join

final_path = "/".join(new_base_join)
print(final_path)

new_base = os.path.join(final_path)
print(new_base)

/home/steven/Documents/aws-data-analytics/notebooks
['', 'home', 'steven', 'Documents', 'aws-data-analytics']
/home/steven/Documents/aws-data-analytics
/home/steven/Documents/aws-data-analytics


## split base path

In [2]:
# print(base_split)

## create new base path

In [3]:
# print(new_base_split)

## join new base path

In [4]:
# print(new_base_join)

## new base path

In [5]:
print(new_base)

/home/steven/Documents/aws-data-analytics


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.column import *
import pandas as pd
import numpy as np
import pprint
import boto3
import time
import json
import ast
import os

with open("./../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

print("imported modules, creds.json.")

imported modules, creds.json.


## pyspark

In [7]:
pp = pprint.PrettyPrinter(indent = 1)
aws_client = creds["aws-access-key"]
aws_secret = creds["aws-secret-access-key"]
spark_host = creds["spark_host"]
spark_port = creds["spark_port"]
extra_jar_list = creds["extra_jar_list"]
print("created initial creds.")

created initial creds.


In [8]:
spark = SparkSession.builder.appName("twitter") \
    .master("spark://{}:{}".format(spark_host, spark_port)) \
    .config("spark.executor.memory", "2048m") \
    .config("spark.executor.cores", "2") \
    .config("spark.streaming.concurrentJobs", "8") \
    .config("spark.local.dir", "/opt/workspace/tmp/driver/") \
    .config("spark.worker.dir", "/opt/workspace/tmp/executor/") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "/opt/workspace/tmp/events/") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.jars.packages", extra_jar_list) \
    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.buffer.dir", "/opt/workspace/tmp/blocks") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext
index = 0
sc.setLogLevel("WARN")
sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
print("imported modules, created spark.")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/steven/.ivy2/cache
The jars for the packages stored in: /home/steven/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e3b24d3-3f6c-43b6-b9ae-bbf6a9267008;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in local-m2-cache
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.32 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in local-m2-cache
	fo

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
s3 = boto3.client("s3")
my_lambda = boto3.client("lambda")
glue = boto3.client("glue")
dynamo = boto3.client("dynamodb")
secretsmanager = boto3.client("secretsmanager")

print("created boto3 clients.")

created boto3 clients.


In [ ]:
creds_keys = creds.keys()
pp.pprint([d for d in creds_keys])

['twitter-api-key',
 'twitter-secret-key',
 'twitter-bearer',
 'twitter-access-token',
 'twitter-secret-access',
 'aws-access-key',
 'aws-secret-access-key',
 'spark_host',
 'spark_port',
 'dynamo_host',
 'dynamo_port',
 'extra_jar_list']


## list aws s3 bucket

list reddit-streaming-stevenhurwitt-new bucket objects.

In [ ]:
response = s3.list_objects_v2(
    Bucket='reddit-streaming-stevenhurwitt-new'
)

## s3 response

In [ ]:
uniq_keys = np.unique([k["Key"].split("/")[0] for k in response["Contents"]])
pp.pprint(uniq_keys)

array(['BikiniBottomTwitter', 'BikiniBottomTwitter_clean',
       'BlackPeopleTwitter', 'BlackPeopleTwitter_clean',
       'ProgrammerHumor'], dtype='<U25')


## read s3 data lake file path

In [ ]:
bucket_name = "reddit-streaming-stevenhurwitt-new"
subreddit = "aws"

In [ ]:
aws_s3_data = "s3a://" + bucket_name + "/" + subreddit + "_clean"
print(aws_s3_data)

s3a://reddit-streaming-stevenhurwitt-new/aws_clean


## read aws_clean delta table w/ pyspark

In [ ]:
df = spark.read().format("delta").load(aws_s3_data)
# df.show()

AttributeError: 'Builder' object has no attribute 'read'

In [ ]:
df.show()

## s3 list objects v2

s3 list objects v2 directory classes (boto3 s3 client).

In [89]:
my_initial_response = s3.list_objects_v2(Bucket = "reddit-streaming-stevenhurwitt-new")
pp.pprint(my_initial_response)

{'Contents': [{'ETag': '"73ec8868d4b7433c3d83d89722685120"',
               'Key': 'BikiniBottomTwitter/_delta_log/00000000000000000000.json',
               'LastModified': datetime.datetime(2023, 1, 29, 21, 59, 52, tzinfo=tzutc()),
               'Size': 9271,
               'StorageClass': 'STANDARD'},
              {'ETag': '"a489b03bcd12ae841aabf8f2139cb346"',
               'Key': 'BikiniBottomTwitter/_delta_log/00000000000000000001.json',
               'LastModified': datetime.datetime(2023, 1, 29, 21, 59, 51, tzinfo=tzutc()),
               'Size': 805,
               'StorageClass': 'STANDARD'},
              {'ETag': '"e42e79b6dffca2de407d5af12df628f7"',
               'Key': 'BikiniBottomTwitter/_delta_log/00000000000000000002.json',
               'LastModified': datetime.datetime(2023, 1, 29, 21, 59, 52, tzinfo=tzutc()),
               'Size': 2227,
               'StorageClass': 'STANDARD'},
              {'ETag': '"e6a3cae3a45dc3bd7ddd259715c65288"',
               'Key

In [49]:
# pp.pprint(response["Contents"])

In [36]:
# ! aws s3 ls s3://reddit-streaming-stevenhurwitt-new